In [120]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import spacy

from tqdm import tqdm
from string import punctuation
from sklearn.metrics import f1_score

from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel

Краткое описание дата сета:

Качество предлагаемого участниками решения будет оцениваться по 2-ум задачам:

1)      Корректное нахождение компаний. Релевантные таблицы:
a.       mentions.csv  - содержит id канала, id сообщения и id упоминаемой компании
b.       mentions_texts.pickle – содержит id канала, id сообщения и текст этого сообщения

2)      Корректное распознавание сентимента. Релевантные таблицы:
a.       sentiment.csv – содержит id канала, id сообщения, id компании и score сентимента
b.       sentiment_texts.pickle - содержит id канала, id сообщения и текст этого сообщения

In [2]:
sns.set_theme()
tqdm.pandas()
nlp = spacy.load('ru_core_news_sm')

c:\Users\sveta\AppData\Local\Programs\Python\Python39\lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'ru_core_news_sm' (3.5.0) was trained with spaCy v3.5.0 and may not be 100% compatible with the current version (3.7.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [3]:
DATA_PATH = 'data'

# Mentions

In [4]:
df = pd.read_pickle(DATA_PATH + '/mentions texts.pickle')
df.head()

,ChannelID,messageid,issuerid,MessageID,DateAdded,DatePosted,MessageText,IsForward
0,1197210433,5408,90,5408,2021-02-06 01:42:42,2020-04-29 07:29:01,?? Фокус недели #ФН Сегодня ????? ММК опублик...,False
1,1203560567,64803,57,64803,2021-02-06 01:47:00,2020-01-21 12:51:42,??#LSRG ЛСР - операционные результаты (2019г)...,False
2,1197210433,23389,152,23389,2021-07-21 13:46:31,2021-07-21 11:15:46,#CHMF Северсталь (CHMF) впервые поставила в Бр...,False
3,1066174394,677,112,677,2021-09-21 04:23:59,2016-12-16 10:00:04,"""Версия: Многоходовочка по Роснефти Роснефтег...",False
4,1239405989,4486,115,4486,2023-01-20 15:02:22,2023-01-20 13:03:38,🟢 Новости к этому часу ⚪️ФРС США необходимо ...,False


Проверим на дубликаты

In [5]:
df['MessageText'].duplicated().sum()

2744

In [6]:
df.duplicated().sum()

0

Следовательно, на некоторые тексты определилось несколько issuerid

In [8]:
df.isna().sum()

ChannelID      0
messageid      0
issuerid       0
MessageID      0
DateAdded      0
DatePosted     0
MessageText    0
IsForward      0
dtype: int64

Предобработка с помощью спейси:

In [10]:
df['spacy'] = df['MessageText'].progress_apply(nlp)

  0%|          | 0/19355 [00:00<?, ?it/s]

100%|██████████| 19355/19355 [29:26<00:00, 10.96it/s] 


In [12]:
df.to_pickle('data/mentions_spacy.pkl')

# Issuers (with synonyms)

In [57]:
issuers_synonyms = pd.read_excel(DATA_PATH + '/names and synonyms.xlsx')
issuers_synonyms.head()

,issuerid,EMITENT_FULL_NAME,VeryOddCompany,BGTicker,BGTicker.1,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,1,"Акционерный коммерческий банк ""Держава"" публич...",NaN,NaN,DERZP,Держава,DERZHAVA,DERZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,"""МОСКОВСКИЙ КРЕДИТНЫЙ БАНК"" (публичное акционе...",NaN,CBOM RX,NaN,Московский кредитный банк,мкб,Credit Bank of Moscow,Credit Bank,NaN,NaN,NaN,NaN,NaN,NaN
2,3,"""Российский акционерный коммерческий дорожный ...",NaN,NaN,RDRB,Российский акционерный коммерческий дорожный банк,РДБанк,Дорожный банк,Russian public joint-stock commercial roads Bank,RosDorBank,РосДорБанк,roads Bank,NaN,NaN,NaN
3,4,"Акционерная компания ""АЛРОСА"" (публичное акцио...",NaN,ALRS RX,NaN,алроса,alrosa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,"Акционерный Коммерческий банк ""АВАНГАРД"" - пуб...",NaN,NaN,AVAN,Авангард,AVANGARD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [102]:
issuers_synonyms[issuers_synonyms['issuerid'] == 90]['EMITENT_FULL_NAME'].values

array(['Публичное акционерное общество "Магнитогорский металлургический комбинат"'],
      dtype=object)

In [58]:
issuers_synonyms.columns[1:]

Index(['EMITENT_FULL_NAME', 'VeryOddCompany', 'BGTicker', 'BGTicker.1',
       'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13',
       'Unnamed: 14'],
      dtype='object')

In [ ]:
synonyms_cols = ['EMITENT_FULL_NAME', 'VeryOddCompany', 'BGTicker', 'BGTicker.1',
       'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14']

In [59]:
issuers_synonyms['count_synonyms'] = (~issuers_synonyms[synonyms_cols].isna()).sum(axis=1)
issuers_synonyms['issuerid_list'] = issuers_synonyms.apply(lambda x: [x['issuerid']] * x['count_synonyms'], axis=1)

In [64]:
synonyms_mapper = pd.DataFrame({
    'company': issuers_synonyms[synonyms_cols].stack().reset_index()[0],
    'issuerid': issuers_synonyms['issuerid_list'].sum()
})

synonyms_mapper

,company,issuerid
0,"Акционерный коммерческий банк ""Держава"" публич...",1
1,DERZP,1
2,Держава,1
3,DERZHAVA,1
4,DERZ,1
...,...,...
1073,"ПАО ""Каршеринг Руссия""",273
1074,"""Каршеринг Руссия"", ПАО",273
1075,Диасофт,274
1076,DIAS,274


In [65]:
synonyms_mapper['spacy'] = synonyms_mapper['company'].progress_apply(nlp)
synonyms_mapper['company_cleaned'] = synonyms_mapper['spacy'].progress_apply(lambda x: ' '.join([t.lemma_ for t in x if t.lemma_ not in punctuation]))
synonyms_mapper.index = synonyms_mapper['company_cleaned']

100%|██████████| 1078/1078 [00:14<00:00, 72.82it/s]


In [74]:
synonyms_mapper[['company', 'issuerid', 'company_cleaned']].to_csv('data/synonyms_mapper.csv')

# Поиск упоминаний компаний

In [109]:
def get_mentions(doc: spacy.language.Doc, synonyms_mapper: pd.DataFrame) -> dict:
    companies = [t.lemma_ for t in doc if t.ent_type_ == 'ORG']
    issuer_ids = synonyms_mapper[synonyms_mapper['company_cleaned'].isin(companies)]['issuerid'].values
    mentions = synonyms_mapper[synonyms_mapper['company_cleaned'].isin(companies)]['company_cleaned'].values
    return {
        'companies': mentions,
        'issuer_ids': issuer_ids
    }

In [110]:
get_mentions(df.loc[0, 'spacy'], synonyms_mapper)

{'companies': array(['норникель', 'ммк', 'нлмк', 'цмт'], dtype=object),
 'issuer_ids': array([ 53,  90, 116, 189], dtype=int64)}

Находим упоминания компаний:

In [79]:
companies = [t.lemma_ for t in df.loc[0, 'spacy'] if t.ent_type_ == 'ORG']
companies

['ммк',
 'фрс',
 'фрс',
 'tesla',
 'microsoft',
 'facebook',
 'мсфо',
 'evraz',
 'норникель',
 'нлмк',
 'apple',
 'amazon',
 'gilead',
 'мосбирже',
 'exxon',
 'mobil',
 'chevron',
 'цмт']

Отбрасываем все, что не входит в список целевых компаний и находим соответствия:

In [88]:
synonyms_mapper[synonyms_mapper['company_cleaned'].isin(companies)][['company',	'issuerid']]

,company,issuerid
company_cleaned,,
норникель,Норникель,53
ммк,ММК,90
нлмк,НЛМК,116
цмт,ЦМТ,189


Индексы для сабмит файла:

In [90]:
found = synonyms_mapper[synonyms_mapper['company_cleaned'].isin(companies)]['issuerid'].values
found

array([ 53,  90, 116, 189], dtype=int64)

# Выделение предложений, в которых упоминается компания

In [141]:
doc = df.loc[0, 'spacy']
mentions = synonyms_mapper[synonyms_mapper['company_cleaned'].isin(companies)]['company_cleaned'].values
sents = [sent.text for company_mention in mentions for sent in doc.sents if company_mention in sent.text.lower()]
sents

['НорНикель опубликует производственные результаты за 1 кв. 2020 #GMKN #Результаты ??????',
 'ММК опубликует финансовую отчётность за 1 кв. 2020 #MAGN #Отчетность ??????',
 'СД НЛМК рассмотрит дивиденды за 1 кв. 2020 #NLMK #Дивиденды  ??????',
 'Акции ЦМТ последний день торгуются с дивидендом 0,56 руб. на акцию #WTCM #Дивиденды']

# Sentiment analysis with dostoevsky

In [121]:
tokenizer = RegexTokenizer()
model = FastTextSocialNetworkModel(tokenizer=tokenizer)

In [130]:
results = model.predict(sents, k=2)

In [133]:
for mention, sentiment in zip(mentions, results):
    print(f'{mention}: {sentiment}')

норникель: {'neutral': 0.9659096002578735, 'skip': 0.0695517510175705}
ммк: {'neutral': 0.8670457601547241, 'skip': 0.09535945951938629}
нлмк: {'neutral': 0.7248802781105042, 'skip': 0.21207880973815918}
цмт: {'neutral': 0.7663036584854126, 'skip': 0.12253321707248688}


In [142]:
results

[{'neutral': 0.9659096002578735, 'skip': 0.0695517510175705},
 {'neutral': 0.8670457601547241, 'skip': 0.09535945951938629},
 {'neutral': 0.7248802781105042, 'skip': 0.21207880973815918},
 {'neutral': 0.7663036584854126, 'skip': 0.12253321707248688}]

# Predict

In [144]:
def predict(doc: spacy.language.Doc, synonyms_mapper: pd.DataFrame, model: FastTextSocialNetworkModel) -> dict:
    companies = [t.lemma_ for t in doc if t.ent_type_ == 'ORG']
    issuer_ids = synonyms_mapper[synonyms_mapper['company_cleaned'].isin(companies)]['issuerid'].values
    mentions = synonyms_mapper[synonyms_mapper['company_cleaned'].isin(companies)]['company_cleaned'].values
    sents = [sent.text for company_mention in mentions for sent in doc.sents if company_mention in sent.text.lower()]
    results = model.predict(sents, k=2)
    
    return {
        'companies': mentions,
        'issuer_ids': issuer_ids,
        'sentiment': results
    }

In [145]:
df['predicts'] = df['spacy'].progress_apply(lambda doc: predict(doc, synonyms_mapper, model))

100%|██████████| 19355/19355 [01:29<00:00, 216.21it/s]


In [151]:
print(f'Предикт на текст, сек: {(60 + 29) / 19355 * 1000}')

Предикт на текст, сек: 4.598295014208215


In [152]:
print(f'Предобработка спейси на текст, сек: {(29 * 60 + 26) / 19355 * 1000}')

Предобработка спейси на текст, сек: 91.2425729785585


In [154]:
print(f'Общее время инференса, сек: {(60 + 29) / 19355 * 1000 + (29 * 60 + 26) / 19355 * 1000}')

Общее время инференса, сек: 95.84086799276672
